In [1]:
import json
import os
import cv2
from multiprocessing import Pool, cpu_count
from concurrent.futures import ThreadPoolExecutor
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


I0000 00:00:1725248151.912880  419551 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1725248151.939887  419551 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1725248151.940046  419551 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


In [3]:
data_dir = './data/dfdc_train_part_02'
metadata_path = os.path.join(data_dir, 'metadata.json')
frames_dir  = os.path.join(data_dir, 'frames')
faces_dir = os.path.join(data_dir, 'cropped_faces')

# Load metadata
with open(metadata_path, 'r') as f:
    metadata = json.load(f)

In [4]:
# Preprocessing - Extract frames from videos

def extract_frames_single_video(args):
    video_path, output_dir, fps = args
    video_name = os.path.splitext(os.path.basename(video_path))[0]
    output_video_dir = os.path.join(output_dir, video_name)
    
    if not os.path.exists(output_video_dir):
        os.makedirs(output_video_dir)
    
    vidcap = cv2.VideoCapture(video_path)
    success, image = vidcap.read()
    count = 0
    while success:
        if count % fps == 0:
            frame_id = int(count / fps)
            cv2.imwrite(os.path.join(output_video_dir, f"frame_{frame_id}.jpg"), image)
        success, image = vidcap.read()
        count += 1

def extract_frames_parallel(video_dir, output_dir, fps=1):
    video_files = [os.path.join(video_dir, f) for f in os.listdir(video_dir) if f.endswith('.mp4')]
    args = [(video, output_dir, fps) for video in video_files]

    with Pool(cpu_count()) as pool:
        pool.map(extract_frames_single_video, args)

extract_frames_parallel(video_dir=data_dir, output_dir=frames_dir, fps=2)